In [1]:
# 80-bit key and 64-bit IV.
# Two 80-bit shift registers, one linear and one non-linear
def h(x):
    hx = x[1] + x[4] + x[0]*x[3] + x[2]*x[3] + x[3]*x[4] + x[0]*x[1]*x[2] + \
    x[0]*x[2]*x[3] + x[0]*x[2]*x[4] + x[1]*x[2]*x[4] + x[2]*x[3]*x[4] + l0 + l0
    return hx

def keystream(S,B):
    z = B[1] + B[2] + B[4] + B[10] + B[31] + B[43] + B[56] + h([S[3], S[25], S[46], S[64], B[63]]) + l0 + l0
    return z

def lfsr_update_fw(S):
    temp = S[62] + S[51] + S[38] + S[23] + S[13] + S[0] + l0 + l0
    
    S = S[1:80] + [temp]
    return S

def lfsr_update_bk(S):
    temp = S[61] + S[50] + S[37] + S[22] + S[12] + S[79] + l0 + l0
    
    S = [temp] + S[:79]
    return S

def nfsr_update_fw(B,s):
    temp = s + B[62] + B[60] + B[52] + B[45] + B[37] + B[33] + B[28] + B[21]\
    + B[14] + B[9] + B[0] + B[63]*B[60] + B[37]*B[33] + B[15]*B[9] + B[60]*B[52]*B[45]\
    + B[33]*B[28]*B[21] + B[63]*B[45]*B[28]*B[9] + B[60]*B[52]*B[37]*B[33] + B[63]*B[60]*B[21]*B[15]\
    + B[63]*B[60]*B[52]*B[45]*B[37] + B[33]*B[28]*B[21]*B[15]*B[9] + B[52]*B[45]*B[37]*B[33]*B[28]*B[21] + l0 + l0
    
    B = B[1:80] + [temp]
    return B

def nfsr_update_bk(B,s):
    temp = s + B[61] + B[59] + B[51] + B[44] + B[36] + B[32] + B[27] + B[20]\
    + B[13] + B[8] + B[79] + B[62]*B[59] + B[36]*B[32] + B[14]*B[8] + B[59]*B[51]*B[44]\
    + B[32]*B[27]*B[20] + B[62]*B[44]*B[27]*B[8] + B[59]*B[51]*B[36]*B[32] + B[62]*B[59]*B[20]*B[14]\
    + B[62]*B[59]*B[51]*B[44]*B[36] + B[32]*B[27]*B[20]*B[14]*B[8] + B[51]*B[44]*B[36]*B[32]*B[27]*B[20] + l0 + l0
    
    B = [temp] + B[:79]
    return B
    
def intersection(A,B):
    temp = []
    for i in range(len(A)):
        if A[i] in B:
            temp.append(A[i])
    return temp

In [ ]:
import time
import csv 
import numpy as np
import gurobipy as gp
from gurobipy import GRB


start2 = time.time()

filename = 'Grain-v1_TMDTO_temp.txt'
csv_file = 'Grainv1_TMDTO_MILP_DATA_temp.csv'

keys = 33
keys_b = 0
keys_f = keys-keys_b


pattern = [0]
for i in range(1,max(keys_f,keys_b)+1,1):
    if i < keys_b+1:
        pattern = pattern + [-i]
    if i < keys_f:
        pattern = pattern + [i]
#pattern = [-i for i in range(1,keys_b+1,1)] + [i for i in range(keys_f)]
#pattern = [i for i in range(-keys_b,0,1)] + [i for i in range(keys_f)]

pattern = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,29,28,27,30,17,31,18,32,19,20,21,22,23,24,25,26]
#pattern = [0, 1, 3, 4, 6, 7, 8, 10, 12, 13, 14, 15, 16, 22, 2, 5, 9, 11, 21, 17, 18, 19, 23, 20, 25, 32, 27, 28, 30, 24, 26, 29, 31]

#S_F_0 = [46, 64, 47, 65, 48, 66, 49, 50, 51, 52, 53, 54, 55, 56, 57, 37, 38, 39, 79, 40, 62,  63]
#B_F_0 = [78]
#S_F_1 = [67, 68, 69, 30, 70, 31, 71, 32, 72, 33, 73, 34, 74, 35, 75, 15, 76, 16, 77, 17, 78]
#S_F_1 = [34, 74, 35, 75, 15, 76, 16, 77, 17, 78]

#S[0] = S[13] + S[38] + S[23] + S[51]


print("Pattern of the keystream bits ", pattern)
state_len = 160
r = [80,80]
V = BooleanPolynomialRing(state_len,['l%d'%i for i in range(r[0])] + ['n%d'%i for i in range(r[1])])
V.inject_variables()
g = list(V.gens())

#for i in S_F_1:
#   g[i] = 1
#g[0] = g[13] + g[38] + g[23] + g[51]
    

S = g[:r[0]]
B = g[r[0]:]


R = [25, 26, 28, 29, 31, 32, 33, 35, 37, 38, 39, 40, 41, 5, 27, 30, 34, 36, 4, 0, 1, 2, 6, 3, 8, 15, 10, 11, 13, 7, 9, 12, 14]
#R = []
F = [i for i in range(state_len) if i not in R]
G = []
for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0


linear_b = [0]*keys_b
linear_f = [0]*keys_f
n_mon = 0
Z_f = [0]*keys_f
Z_b = [0]*keys_b

for loop in range(keys_b):
    S = lfsr_update_bk(S)
    B = nfsr_update_bk(B,S[0])

    Z_b[loop] = keystream(S,B)
    monom = list(Z_b[loop].monomials())
    linear_b[loop] = [m for m in monom if m.degree() == 1]
    for m in monom:
        if m.degree()>1:
            n_mon += 1
    #print("backward loop 1st part ", loop)
    
S = g[:r[0]]
B = g[r[0]:]

for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0


for loop in range(keys_f):
    Z_f[loop] = keystream(S,B)
    monom = list(Z_f[loop].monomials())
    linear_f[loop] = [m for m in monom if m.degree() == 1]
    for m in monom:
        if m.degree()>1:
            n_mon += 1
    
    #print("forward loop 1st part ",loop)
    if loop != keys_f-1:
        B=nfsr_update_fw(B,S[0])
        S=lfsr_update_fw(S) 
#print("1st part done")


index = 0
Z = [0]*(keys_f+keys_b)
linear = [0]*(keys_f + keys_b)
linear_new = [0]*(keys_f + keys_b)
linear_all = []

for i in pattern:
    if i < 0:
        Z[index] = Z_b[-i-1]
        linear[index] = linear_b[-i-1]
    if i >= 0:
        Z[index] = Z_f[i]
        linear[index] = linear_f[i]
    index += 1

for loop in range(len(Z)):
    lin = []
    for i in linear[loop]:
        if g.index(i) not in linear_all:
            lin = lin + [g.index(i)]
            
    linear_new[loop] = lin
    linear_all = linear_all + lin
    
#########Gurobi Modelling Starts###############
model = gp.Model()


v = [0,1,2]             # Temporary variable to be used for gurobi variable definition
aux = [i for i in range(n_mon)]

#Let D, F, G represents the class of determined/recovery, Fixed bit, and Guess bit
#Since these are set and we need variables to denote R,F and G therefore let 2=R, 3=F and 4=G
#x represents variables for each register and m contains 3 variables for each x[i] i.e. for R,F and G
x = model.addVars(len(S)+len(B), lb = 2, ub = 4, vtype = GRB.INTEGER, name = "x")
m = model.addVars(len(S)+len(B), len(v), vtype = GRB.BINARY, name = "m")
q = model.addVars(len(aux), vtype = GRB.BINARY, name = "q")
#p = model.addVars(len(aux), vtype = GRB.BINARY, name = "p")
#r = model.addVars(len(aux), vtype = GRB.BINARY, name = "r")
model.update()

model.addConstrs((m[i,0] == 1) for i in R)
model.addConstrs((m[i,1] == 1) for i in F)
model.addConstrs((m[i,2] == 1) for i in G)

#Objective function is to maximize recovery bit and minimize the number of FIXED bit
model.setObjectiveN(gp.quicksum((m[i,0]) for i in range(state_len)),0,1, GRB.MAXIMIZE)
model.setObjectiveN(gp.quicksum((m[i,1]) for i in range(state_len)),1,0, GRB.MINIMIZE)


model.addConstrs((x[i] == (2*m[i,0] + 3*m[i,1] + 4*m[i,2])) for i in range(state_len))
model.addConstrs(m[i,0] + m[i,1] + m[i,2] == 1 for i in range(state_len))

#Number of recovered bit should be less than or equal to the number of keystream equations
model.addConstr(gp.quicksum(m[i,0] for i in range(state_len)) == keys)
model.addConstr(gp.quicksum(m[i,0] for i in range(state_len) if i not in linear_all) == 0)

count = 0
cnt = 0
for loop in range(keys_f + keys_b):
    z = Z[loop]
    #z_var contains the index of variables in Z
    #z_var = np.array(z.variables())
    #z_var = [g.index(i) for i in z_var]
    z_monom = np.array(z.monomials())

    z_linear = [g.index(i) for i in z_monom if i.degree() == 1]

    #z_nlinear = [list(i.variables()) for i in z_monom if len(i.variables()) > 1]
    #z_nlinear = [g.index(z_nlinear[i][j]) for i in range(len(z_nlinear)) for j in range(len(z_nlinear[i]))]
    #z_nlinear = list(set(z_nlinear))


    #linear_new = [i for i in z_linear if i not in (linear + z_nlinear)]
    #z_linear_org = [i for i in z_linear if i not in z_nlinear]


    #One of the linear term variables which is not in the previous linear term has one recovered bit
    model.addConstr(gp.quicksum(m[i,0] for i in linear_new[loop]) <= 1)    
    #model.addConstr(gp.quicksum(m[i,0] for i in z_linear) <= loop + 1) 


    if loop <= 33:
        for mon in z_monom:
            if mon.degree() > 1:
                var = list(mon.variables())
                var = [g.index(i) for i in var]
                inters = [0]*len(linear_new)
                n = [0]*len(linear_new)
                for l in range(len(linear_new)):
                    inters[l] = intersection(var,linear_new[l])
                    n[l] = len(inters[l])

                C_prev = [n[i] for i in range(loop)] 
                C_current = [n[loop]]
                C_future = [n[i] for i in range(loop+1,keys,1)]

                #Case 1: recovered var in mon and is in linear
                #if sum(C_prev) > 0 and sum(C_current) == 0 and sum(C_future) == 0 :
                #    #if recovery found then guess
                #    model.addConstr((p[count] == 0) >> (gp.quicksum(m[i,0] for i in var) == 0))
                #    model.addConstr((p[count] == 1) >> (gp.quicksum((m[i,1] + m[i,2]) for i in var) >= len(var) - sum(C_prev)))
                #    count += 1

                #Case 2: recovered var in mon and is in z_linear-linear
                if (sum(C_current + C_future) > 0):
                    #if recovery found then fix to zero
                    prev = []
                    for k in range(loop):
                        prev = prev + inters[k]
                    #var_ge = [i for i in var if i not in prev]

                    var_ge = []
                    for k in range(loop,len(linear_new),1):
                        var_ge = var_ge + inters[k]

                    model.addConstr((q[count] == 0) >> (gp.quicksum(m[i,0] for i in var_ge) == 0))
                    model.addConstr((q[count] == 1) >> (gp.quicksum(m[i,1] for i in var) >= 1))
                    count += 1

                #if (sum(C_prev + C_current + C_future) == 0): 
                #    model.addConstr(gp.quicksum((m[i,0]) for i in var) == 0)
        print("2nd part loop ", loop)
print("2nd part done")
import time

model.params.outputflag = 0
start = time.time()
model.optimize()
end = time.time() - start

print("Keystream = ",keys)
print("status",model.STATUS)

f = open(filename,'a')
f.write("\nKeystream = " + str(keys))
f.write("\nStatus " + str(model.STATUS) + "\n")
f.write("Pattern = " + str(pattern))

if model.STATUS == 2:
    val = model.x
    R = []
    F = []
    G = []
    for i in range(state_len):
        if round(val[i]) == 2:
            R.append(i)
        if round(val[i]) == 3:
            F.append(i)
        if round(val[i]) == 4:
            G.append(i)



    Rec = [g[i] for i in R]
    Fix = [g[i] for i in F]
    Guess = [g[i] for i in G]
    print("Recovery Bit ",Rec)
    print("Fixed Bit ",Fix)
    print("Guessed Bit ",Guess)
    
    f.write(str("Recovery Bit ") + str(Rec) + "\n")
    f.write(str("Fixed Bit ") + str(Fix) + "\n")
    f.write(str("Guessed Bit ") + str(Guess) + "\n")
    
    print(len(R),len(F))
    if (len(R) + len(F) + len(G) == state_len) and len(intersection(R,F) + intersection(F,G) + intersection(R,G)) == 0:
        print("Length verified")
        f.write("\nLength Verified\n\n")
end2 = time.time()-start2
print("Gurobi Time = ", end)
print("Overall Time = ", end2)
f.close()

In [ ]:
temp = []
for i in range(len(linear_new)):
    temp += [linear_new[i][0]]
print(temp)
print(linear_new)

In [2]:
state_len = 160
r = [80,80]
V = BooleanPolynomialRing(state_len,['l%d'%i for i in range(r[0])] + ['n%d'%i for i in range(r[1])])
V.inject_variables()
g = list(V.gens())

Defining l0, l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11, l12, l13, l14, l15, l16, l17, l18, l19, l20, l21, l22, l23, l24, l25, l26, l27, l28, l29, l30, l31, l32, l33, l34, l35, l36, l37, l38, l39, l40, l41, l42, l43, l44, l45, l46, l47, l48, l49, l50, l51, l52, l53, l54, l55, l56, l57, l58, l59, l60, l61, l62, l63, l64, l65, l66, l67, l68, l69, l70, l71, l72, l73, l74, l75, l76, l77, l78, l79, n0, n1, n2, n3, n4, n5, n6, n7, n8, n9, n10, n11, n12, n13, n14, n15, n16, n17, n18, n19, n20, n21, n22, n23, n24, n25, n26, n27, n28, n29, n30, n31, n32, n33, n34, n35, n36, n37, n38, n39, n40, n41, n42, n43, n44, n45, n46, n47, n48, n49, n50, n51, n52, n53, n54, n55, n56, n57, n58, n59, n60, n61, n62, n63, n64, n65, n66, n67, n68, n69, n70, n71, n72, n73, n74, n75, n76, n77, n78, n79


In [5]:
import time
pattern = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,29,28,27,30,17,31,18,32,19,20,21,22,23,24,25,26]
start2 = time.time()

filename = 'Grain-v1_TMDTO_temp.txt'
csv_file = 'Grainv1_TMDTO_MILP_DATA_temp.csv'

keys = 33
keys_b = 0
keys_f = keys-keys_b

print("Pattern of the keystream bits ", pattern)


R = [80+i for i in range(10,26,1)] + [80 + i for i in [79, 30, 29, 28, 73, 74, 75]] + [42, 43, 44, 45, 18, 19, 58, 59, 60, 61]
F = [15, 16, 74, 76, 77, 46, 64, 47, 65, 48, 66, 49, 50, 51, 52, 53, 54, 55, 56, 57, 37, 38, 39, 79, 40, 62, 63, 80+78]
F = []
G = []

Rec = [g[i] for i in R]
print("Recovered Bits are ", Rec)
Fix = [g[i] for i in F]
print("Fixed Bits are ", Fix)


#Assigning variables to the registers
S = g[:r[0]]
B = g[r[0]:]

for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0

        
Z_f = [0]*keys_f
Z_b = [0]*keys_b

for loop in range(keys_b):
    S = lfsr_update_bk(S)
    B = nfsr_update_bk(B,S[0])

    Z_b[loop] = keystream(S,B)
    
    
S = g[:r[0]]
B = g[r[0]:]

for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0

for loop in range(keys_f):
    Z_f[loop] = keystream(S,B)
    
    if loop != keys_f-1:
        B=nfsr_update_fw(B,S[0])
        S=lfsr_update_fw(S) 


index = 0
Z = [0]*(keys_f+keys_b)

for i in pattern:
    if i < 0:
        Z[index] = Z_b[-i-1]
    if i >= 0:
        Z[index] = Z_f[i]
    index += 1


R_past = []
L_rec = []
cnt = 0
flag = 1
for i in range(keys_f+keys_b):
    print(i)
    z = Z[i]
    monom = list(z.monomials())
    t_var = list(z.variables())

    R_past = R_past + list(set(L_rec))
    m_rec = []
    for m in monom:
        m_var = list(m.variables())
        if (len(m_var) == 1) and (len(intersection(m_var,Rec)) > 0):
            L_rec = L_rec + m_var

        if (len(m_var) > 1) and (len(intersection(m_var,Rec)) > 0):
            m_rec = m_rec + [m]
    #print("monomials containing recovery bits (degree > 1) ",m_rec)

    for m in m_rec:
        m_var = list(m.variables())
        #Now L_rec contains current and past recovery bit in linear terms
        #Whereas R_past contains only the past recovery bit in linear terms
        
        if len(intersection(m_var,Rec)) > len(intersection(m_var,R_past))\
        and len(intersection(m_var,Fix)) == 0:
            f = open(filename,'a')
            f.write("\nError --- Future recovery bits found\n" + str(m))
            f.close()
            print(m,"(",intersection(m_var,Rec),")",end = " + ")
            flag = 0
            

    L_rec = list(set(L_rec))
    if len(L_rec) != cnt + 1:
        flag = 0
    cnt = len(L_rec)
    print("\nRecovery bits till now in linear terms \n",L_rec)

f = open(filename,'a')
if flag == 0:
    print("Failed")
    f.write("\nFailed\n")
else:
    print("Passed")
    f.write("\nPassed\n")
    with open(csv_file,'a') as f:
        writer = csv.writer(f)
        #writer.writerow(['Number of Recovered bits','Number of Fixed bits','Recovered bits','Fixed bits'])
        writer.writerow([len(R),len(F),Rec,Fix])
f.close()


Pattern of the keystream bits  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 29, 28, 27, 30, 17, 31, 18, 32, 19, 20, 21, 22, 23, 24, 25, 26]
Recovered Bits are  [n10, n11, n12, n13, n14, n15, n16, n17, n18, n19, n20, n21, n22, n23, n24, n25, n79, n30, n29, n28, n73, n74, n75, l42, l43, l44, l45, l18, l19, l58, l59, l60, l61]
Fixed Bits are  []
[l0, l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11, l12, l13, l14, l15, l16, l17, l18, l19, l20, l21, l22, l23, l24, l25, l26, l27, l28, l29, l30, l31, l32, l33, l34, l35, l36, l37, l38, l39, l40, l41, l42, l43, l44, l45, l46, l47, l48, l49, l50, l51, l52, l53, l54, l55, l56, l57, l58, l59, l60, l61, l62, l63, l64, l65, l66, l67, l68, l69, l70, l71, l72, l73, l74, l75, l76, l77, l78, l79]
0

Recovery bits till now in linear terms 
 [n10]
1

Recovery bits till now in linear terms 
 [n10, n11]
2

Recovery bits till now in linear terms 
 [n10, n11, n12]
3

Recovery bits till now in linear terms 
 [n10, n13, n11, n12]
4

Recovery bits til

l8*l60*l71 ( [l60] ) + l8*l60 ( [l60] ) + l28*l60*l71 ( [l60] ) + l28*l60 ( [l60] ) + l60*l71*n8 ( [l60] ) + l60*l71*n17*n23*n29*n36*n41 ( [l60, n17, n23, n29] ) + l60*l71*n17*n23 ( [l60, n17, n23] ) + l60*l71*n17*n36*n53*n71 ( [l60, n17] ) + l60*l71*n17 ( [l60, n17] ) + l60*l71*n22 ( [l60, n22] ) + l60*l71*n23*n29*n68*n71 ( [l60, n23, n29] ) + l60*l71*n29*n36*n41*n45*n53*n60 ( [l60, n29] ) + l60*l71*n29*n36*n41 ( [l60, n29] ) + l60*l71*n29 ( [l60, n29] ) + l60*l71*n36 ( [l60] ) + l60*l71*n41*n45*n60*n68 ( [l60] ) + l60*l71*n41*n45 ( [l60] ) + l60*l71*n41 ( [l60] ) + l60*l71*n45*n53*n60*n68*n71 ( [l60] ) + l60*l71*n45 ( [l60] ) + l60*l71*n53*n60*n68 ( [l60] ) + l60*l71*n53 ( [l60] ) + l60*l71*n60 ( [l60] ) + l60*l71*n68*n71 ( [l60] ) + l60*l71*n68 ( [l60] ) + l60*l71*n70 ( [l60] ) + l60*l71 ( [l60] ) + l60*n8 ( [l60] ) + l60*n17*n23*n29*n36*n41 ( [l60, n17, n23, n29] ) + l60*n17*n23 ( [l60, n17, n23] ) + l60*n17*n36*n53*n71 ( [l60, n17] ) + l60*n17 ( [l60, n17] ) + l60*n22 ( [l60, n22]

In [12]:
Rec.sort()
Rec.reverse()
print(Rec)

[l18, l19, l42, l43, l44, l45, l58, l59, l60, l61, n10, n11, n12, n13, n14, n15, n16, n17, n18, n19, n20, n21, n22, n23, n24, n25, n28, n29, n30, n73, n74, n75, n79]


In [9]:
A = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,29,28,27,30,17,31,18,32,19,20,21,22,23,24,25,26]

S_F_0 = [46, 64, 47, 65, 48, 66, 49, 50, 51, 52, 53, 54, 55, 56,\
         57, 37, 38, 39, 79, 40, 62,  63]
B_F_0 = [78]
S_F_1 = [67, 68, 69, 30, 70, 31, 71, 32, 72, 33, 73, 34, 74, 35, \
         75, 15, 76, 16, 77, 17, 78]

S[0] = S[13] + S[38] + S[23] + S[51]

S_F_0.sort()
S_F_1.sort()
print("Fixed to Zero ")
for i in S_F_0:
    print(g[i],end=", ")
for i in B_F_0:
    print(g[80+i])
print("\nFixed to 1")
for i in S_F_1:
    print(g[i],end = ", ")


Fixed to Zero 
l37, l38, l39, l40, l46, l47, l48, l49, l50, l51, l52, l53, l54, l55, l56, l57, l62, l63, l64, l65, l66, l79, n78

Fixed to 1
l15, l16, l17, l30, l31, l32, l33, l34, l35, l67, l68, l69, l70, l71, l72, l73, l74, l75, l76, l77, l78, 

In [ ]:
Fixed Bits to 0 
LFSR S
S[46], S[64], S[47], S[65], S[48], S[66], S[49], S[50], S[51], S[52], S[53], S[54] (+2), S[55],
S[56], S[57], S[37], S[38], S[39], S[79], S[40], S[62],  S[63]

NFSR B
B[78]


Fixed Bits to 1
LFSR S
S[67], S[68], S[69], S[30], S[70], S[31], S[71], S[32], S[72], S[33], S[73], S[34], S[74], S[35], S[75], S[15], S[76], S[16], S[77], S[17], S[78]

NFSR B




S[0] = S[13] + S[38] + S[23] + S[51]


In [26]:
print(log(1099511627776,2))

40


In [32]:
log(1<<40,2)

40